### 머신러닝을 위용한 텍스트 감성 분류

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
file_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/cleaned_df.csv'

df = pd.read_csv(file_path)
df

,review,sentiment
0,stuff going moment mj started listening music ...,1
1,classic war worlds timothy hines entertaining ...,1
2,film starts manager nicholas bell giving welco...,0
3,must assumed praised film greatest filmed oper...,0
4,superbly trashy wondrously unpretentious explo...,1
...,...,...
24995,seems like consideration gone imdb reviews fil...,0
24996,believe made film completely unnecessary first...,0
24997,guy loser get girls needs build picked stronge...,0
24998,minute documentary bu uel made early one spain...,0


#### CountVectorizer 모델을 활용한 문장 임베딩

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
reviews = df.loc[:, 'review'].values
print(reviews)

['stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate worki

In [ ]:
count = CountVectorizer(max_features = 10000)

In [ ]:
### 단어 사전 생성
count.fit(reviews)

CountVectorizer(max_features=10000)

In [ ]:
### 생성된 단어 사전 확인

# 단어 사전 추출
vocab = count.vocabulary_

# 결과 확인하기
print(f'단어의 수 : {len(vocab)}')
print('-'*80)
print(vocab)
print('-'*80)

print(sorted(vocab.items()))

단어의 수 : 10000
--------------------------------------------------------------------------------
{'stuff': 8626, 'going': 3836, 'moment': 5788, 'mj': 5766, 'started': 8470, 'listening': 5235, 'music': 5900, 'watching': 9707, 'odd': 6170, 'documentary': 2616, 'watched': 9704, 'maybe': 5560, 'want': 9668, 'get': 3761, 'certain': 1400, 'insight': 4580, 'guy': 3989, 'thought': 9017, 'really': 7169, 'cool': 1955, 'eighties': 2859, 'make': 5418, 'mind': 5702, 'whether': 9783, 'guilty': 3975, 'innocent': 4568, 'part': 6409, 'biography': 878, 'feature': 3318, 'film': 3386, 'remember': 7310, 'see': 7832, 'cinema': 1555, 'originally': 6267, 'released': 7281, 'subtle': 8665, 'messages': 5650, 'feeling': 3329, 'towards': 9156, 'press': 6825, 'also': 267, 'obvious': 6155, 'message': 5649, 'drugs': 2729, 'bad': 659, 'kay': 4888, 'visually': 9605, 'impressive': 4463, 'course': 2014, 'michael': 5668, 'jackson': 4739, 'unless': 9414, 'remotely': 7321, 'like': 5192, 'anyway': 387, 'hate': 4090, 'find': 34

In [ ]:
### transform(text).toarray() --> 리뷰 --> 임베딩
X_data = count.transform(reviews).toarray()
print(f'리뷰를 임베딩한 결과(모양) : {X_data.shape}')
print('-'*80)
print(f'첫번째 리뷰에 대한 임베딩의 결과 : \n{X_data[0, :]}')

리뷰를 임베딩한 결과(모양) : (25000, 10000)
--------------------------------------------------------------------------------
첫번째 리뷰에 대한 임베딩의 결과 : 
[0 0 0 ... 0 0 0]


In [ ]:
df.loc[:, 'sentiment'].value_counts(normalize = True)

sentiment
1    0.5
0    0.5
Name: proportion, dtype: float64

#### train test 데이터 나누기

In [ ]:
from sklearn.model_selection import train_test_split

# X_data는 위에서 생성

# y_data --> sentiment 칼럼
y_data = df.loc[:, 'sentiment']

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.2, random_state = 0, stratify = y_data)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(20000, 10000) (20000,)
(5000, 10000) (5000,)


In [ ]:
print(np.unique(y_train, return_counts = True))

(array([0, 1]), array([10000, 10000]))


#### RandomForest 분류 모델을 이용한 텍스트 감성 분석

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 500, random_state = 0)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=500, random_state=0)

In [ ]:
##### 평가용 데이터를 이용한 예측
y_pred = model.predict(X_test)
print(y_pred)

[0 0 0 ... 1 1 1]


##### 모델 평가

In [ ]:
from sklearn.metrics import classification_report

performance = classification_report(y_test, y_pred)

print(performance)

              precision    recall  f1-score   support

           0       0.86      0.85      0.85      2500
           1       0.85      0.86      0.86      2500

    accuracy                           0.85      5000
   macro avg       0.86      0.85      0.85      5000
weighted avg       0.86      0.85      0.85      5000



In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

0.855
0.8576
0.8531635495423796
0.855376022341911


#### LightGBM 분류 모델을 이용한 텍스트 감성 분석

In [ ]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(learning_rate = 0.1, n_estimators = 500, random_state = 0,
                      bagging_seed = 0, feature_fraction_seed = 0, data_random_seed = 0,
                      drop_seed = 0, verbose = -1)

In [ ]:
lgbm.fit(X_train, y_train)

LGBMClassifier(bagging_seed=0, data_random_seed=0, drop_seed=0,
               feature_fraction_seed=0, n_estimators=500, random_state=0,
               verbose=-1)

In [ ]:
y_pred = lgbm.predict(X_test)
print(y_pred)

[1 0 0 ... 1 1 1]


##### 모델평가

In [ ]:
perform = classification_report(y_test, y_pred)
print(perform)

              precision    recall  f1-score   support

           0       0.89      0.87      0.88      2500
           1       0.87      0.89      0.88      2500

    accuracy                           0.88      5000
   macro avg       0.88      0.88      0.88      5000
weighted avg       0.88      0.88      0.88      5000



### Word2Vec 모델을 이용한 리뷰 데이터 임베딩

In [3]:
from gensim.models.word2vec import Word2Vec

In [4]:
### review 칼럼 --> 단어(토큰) 단위로 분할 --> 리스트 데이터 생성

# 결과 저장용 리스트 생성
sentences = []

# for문 + split() + append() 함수 사용
for review in df.loc[:, 'review']:
  #print(review)
  #print('-'*80)
  token_list = review.split()
  #print(token_list)
  #print('-'*80)
  sentences.append(token_list)

print(sentences[0])

['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker', 'maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema', 'originally', 'released', 'subtle', 'messages', 'mj', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'mj', 'anyway', 'going', 'hate', 'find', 'boring', 'may', 'call', 'mj', 'egotist', 'consenting', 'making', 'movie', 'mj', 'fans', 'would', 'say', 'made', 'fans', 'true', 'really', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding', 'smooth', 'criminal', 'sequence', 'joe', 'pesci', 'convincing', 'psychopathic', 'powerful', 'drug', 'lord', 

In [62]:
### skip-gram 모델 학습 --> 단어 사전 + 임베딩 벡터 생성
sg = Word2Vec(sentences = sentences, vector_size = 300, window = 10, min_count = 10, workers = 1, sg = 1)

In [63]:
### 모델 저장하기
#save_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/sg.model'
save_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/sg1.model'
sg.save(save_path)

In [65]:
#save_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/sg.model'
save_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/sg1.model'
loaded_sg = Word2Vec.load(save_path)

In [66]:
vocab = loaded_sg.wv.key_to_index
print(vocab)
print(len(vocab))

{'movie': 0, 'film': 1, 'one': 2, 'like': 3, 'good': 4, 'time': 5, 'even': 6, 'would': 7, 'story': 8, 'really': 9, 'see': 10, 'well': 11, 'much': 12, 'get': 13, 'bad': 14, 'people': 15, 'also': 16, 'first': 17, 'great': 18, 'made': 19, 'way': 20, 'make': 21, 'could': 22, 'movies': 23, 'think': 24, 'characters': 25, 'character': 26, 'watch': 27, 'two': 28, 'films': 29, 'seen': 30, 'many': 31, 'life': 32, 'plot': 33, 'acting': 34, 'never': 35, 'love': 36, 'little': 37, 'best': 38, 'show': 39, 'know': 40, 'ever': 41, 'man': 42, 'better': 43, 'end': 44, 'still': 45, 'say': 46, 'scene': 47, 'scenes': 48, 'go': 49, 'something': 50, 'back': 51, 'real': 52, 'watching': 53, 'though': 54, 'thing': 55, 'old': 56, 'years': 57, 'actors': 58, 'director': 59, 'work': 60, 'another': 61, 'new': 62, 'nothing': 63, 'funny': 64, 'actually': 65, 'makes': 66, 'look': 67, 'find': 68, 'going': 69, 'part': 70, 'lot': 71, 'every': 72, 'world': 73, 'cast': 74, 'us': 75, 'quite': 76, 'got': 77, 'want': 78, 'thing

In [67]:
print(loaded_sg.wv['movie'])

[ 1.74954131e-01  8.59208703e-02  3.53843421e-02  1.51214842e-02
 -1.91871047e-01 -1.86396703e-01  2.42117956e-01  1.89295128e-01
 -6.61567748e-02 -1.15021072e-01  2.44801968e-01  1.11205177e-02
  2.45363135e-02 -4.75049317e-02 -4.23241518e-02  1.23994380e-01
  1.50531486e-01 -7.22108930e-02 -7.60310143e-02 -7.65914023e-02
  2.18829531e-02  4.05713059e-02  2.20117167e-01  1.85203943e-02
 -1.60850678e-02 -5.34102581e-02  5.47484197e-02 -1.98190883e-01
 -1.58741727e-01 -1.08619601e-01 -9.37326029e-02 -1.23356981e-03
 -7.62472451e-02 -1.80507332e-01 -1.47783518e-01  1.29222736e-01
 -1.70696467e-01 -2.09777281e-01  9.88823827e-03 -1.71876684e-01
 -1.27992406e-01 -1.67274717e-02  1.50894031e-01 -1.70885980e-01
  1.49132103e-01  1.71547964e-01  3.07749528e-02  1.02807574e-01
 -8.08485448e-02  1.92883089e-02  2.40383103e-01 -2.58262753e-02
 -6.95386752e-02 -1.57702789e-02 -1.39449656e-01  6.30413136e-03
  3.64639536e-02  7.39220008e-02  1.91088300e-03 -5.89750819e-02
 -8.50444362e-02  1.42711

In [70]:
print(list(loaded_sg.wv.key_to_index.keys()))

['movie', 'film', 'one', 'like', 'good', 'time', 'even', 'would', 'story', 'really', 'see', 'well', 'much', 'get', 'bad', 'people', 'also', 'first', 'great', 'made', 'way', 'make', 'could', 'movies', 'think', 'characters', 'character', 'watch', 'two', 'films', 'seen', 'many', 'life', 'plot', 'acting', 'never', 'love', 'little', 'best', 'show', 'know', 'ever', 'man', 'better', 'end', 'still', 'say', 'scene', 'scenes', 'go', 'something', 'back', 'real', 'watching', 'though', 'thing', 'old', 'years', 'actors', 'director', 'work', 'another', 'new', 'nothing', 'funny', 'actually', 'makes', 'look', 'find', 'going', 'part', 'lot', 'every', 'world', 'cast', 'us', 'quite', 'got', 'want', 'things', 'pretty', 'young', 'seems', 'around', 'horror', 'however', 'fact', 'take', 'big', 'enough', 'long', 'thought', 'series', 'may', 'give', 'original', 'action', 'right', 'without', 'must', 'comedy', 'always', 'times', 'point', 'gets', 'family', 'come', 'role', 'saw', 'almost', 'interesting', 'least', 'do

In [72]:
### review --> 문장 1개 --> 임베딩 벡터 추출 함수 정의

def get_features(sentence, model):
  # 각 단어 임베딩 벡터를 저장할 리스트 생성
  sentence_embedding = []
  # 단어 사전 --> key(word) 리스트 생성
  key_list = list(model.wv.key_to_index.keys())
  # for문 + append()
  for word in sentence:
    if word in key_list:
      word_vec = model.wv[word]
      sentence_embedding.append(word_vec)
  # 단어 임베딩 결과 --> 데이터 프레임 생성
  df_sent = pd.DataFrame(data = sentence_embedding)
  # 각 칼럼별 평균 값 추출
  embedding_mean = df_sent.mean().values
  # 각 칼럼별 중간 값 추출
  embedding_median = df_sent.median().values
  # 각 칼럼별 최대 값 추출
  embedding_max = df_sent.max().values

  return embedding_mean, embedding_median, embedding_max

In [73]:
### get_features 함수 테스트
embedding_mean, embedding_median, embedding_max = get_features(sentence = sentences[0], model = loaded_sg)
print(embedding_max)

[0.5172681  0.65964097 0.3226557  0.35795832 0.4839524  0.20414564
 0.54849356 0.7028726  0.6007286  0.33317962 0.4988438  0.23815149
 0.47756824 0.2551721  0.32227427 0.4406187  0.47991768 0.3096368
 0.40630597 0.5230851  0.33351752 0.42988986 0.7051021  0.4249369
 0.6193487  0.47124904 0.2824113  0.4163456  0.35757166 0.17766277
 0.40600786 0.5567693  0.25057918 0.3505231  0.3929027  0.43380922
 0.38666672 0.29291072 0.4554401  0.3570322  0.40744388 0.3967902
 0.29911107 0.32720566 0.48454988 0.4482122  0.5231575  0.47267765
 0.48250034 0.51548076 0.5957285  0.44908828 0.40993825 0.32728198
 0.34115696 0.47655946 0.412117   0.38693154 0.43317813 0.2687177
 0.40928102 0.36833444 0.42514893 0.45926002 0.34726515 0.38579828
 0.31885788 0.3633106  0.4160033  0.43430346 0.31190354 0.417824
 0.46223643 0.33023983 0.54464644 0.30916598 0.3883167  0.53257376
 0.3332177  0.63884085 0.35945746 0.23655032 0.36936915 0.56820977
 0.50743186 0.3317911  0.41173413 0.35906142 0.5509591  0.5247689
 0

In [74]:
### 전체 리뷰에 대한 임베딩 행렬(25000, 300) 생성 함수 정의

# 진행 경과 표시하기 위한 함수 임폴트
from tqdm import tqdm

def get_dataset(sentences, model = loaded_sg):
  # 전체 리뷰의 임베딩 벡터를 저장할 리스트 생성
  sentences_embedding_mean = []
  sentences_embedding_median = []
  sentences_embedding_max = []
  # for문 + append()
  for sentence in tqdm(sentences):
    mean, median, max = get_features(sentence, model)
    sentences_embedding_mean.append(mean)
    sentences_embedding_median.append(median)
    sentences_embedding_max.append(max)
  # 전체 리뷰 임베딩 결과 --> 데이터프레임
  df_mean = pd.DataFrame(data =  sentences_embedding_mean)
  df_median = pd.DataFrame(data =  sentences_embedding_median)
  df_max = pd.DataFrame(data =  sentences_embedding_max)
  # 데이터프레임 --> 값 추출 --> 2차원 배열 생성(25000, 300)
  embeddings_mean = df_mean.values
  embeddings_median = df_median.values
  embeddings_max = df_max.values

  return embeddings_mean, embeddings_median, embeddings_max

In [75]:
### get_dataset 함수 실행 --> 전체 리뷰에 대한 임베딩 행렬 생성
embeddings_mean, embeddings_median, embeddings_max = get_dataset(sentences = sentences)


100%|██████████| 25000/25000 [12:47<00:00, 32.55it/s]


In [76]:
print(embeddings_mean.shape)
print(type(embeddings_mean))

(25000, 300)
<class 'numpy.ndarray'>


In [77]:
### 생성된 임베딩 행렬 저장

# 파일 경로 설정
mean_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/embeddings_mean.npy'
median_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/embeddings_median.npy'
max_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/embeddings_max.npy'

# 저장 : 배열.save(path)
np.save(mean_path, embeddings_mean)
np.save(median_path, embeddings_median)
np.save(max_path, embeddings_max)

In [78]:
### 저장된 임베딩 행렬 불러오기

# np.load(path) 함수 사용
embeddings_mean = np.load(mean_path)
embeddings_median = np.load(median_path)
embeddings_max = np.load(max_path)

print(embeddings_mean[0, :])

[ 0.08597377  0.15448263 -0.05750388 -0.06573609  0.01380978 -0.13389674
  0.1540865   0.23906873  0.05010077 -0.07055769  0.0439544  -0.02673463
  0.04438124 -0.06707734 -0.04439891  0.03516045  0.04029266 -0.03793162
 -0.00934309 -0.00739933 -0.04458098 -0.00286708  0.18144563  0.03760145
  0.08316305  0.02696406 -0.02842201  0.01598717 -0.06514984 -0.13094398
 -0.03114918  0.00336448 -0.01911533 -0.07177798 -0.04940608  0.06275205
 -0.0707102  -0.07691586  0.0335569   0.01919793 -0.0228146  -0.02831204
 -0.03152225 -0.04025183  0.0304944   0.11575687  0.02344193  0.00110341
  0.00626853  0.08576621  0.14105612  0.00891486 -0.12025186 -0.01428064
 -0.03877844  0.09775444  0.07453221 -0.02178605  0.03562494 -0.11532521
 -0.08360589  0.01423168  0.02253352  0.03836526  0.00900798 -0.01953556
 -0.05823145  0.01434139 -0.06182426 -0.00701151 -0.0255726   0.14211926
  0.06831564 -0.04452013 -0.03910628 -0.0032452  -0.01705823  0.00868544
  0.0166449   0.03647259 -0.03293843 -0.06026194 -0

#### 학습용 / 평가용 데이터 생성

In [80]:
from sklearn.model_selection import train_test_split

# X_data, y_data 생성
X_data_mean = embeddings_mean[:,:]
X_data_median = embeddings_median[:,:]
X_data_max = embeddings_max[:,:]

y_data = df.loc[:, 'sentiment'].values

# 학습용 / 평가용으로 분할
X_train_mean, X_test_mean, y_train, y_test = train_test_split(X_data_mean, y_data, test_size = 0.2,
                                                              random_state = 0, stratify = y_data)

X_train_median, X_test_median, y_train, y_test = train_test_split(X_data_median, y_data, test_size = 0.2,
                                                              random_state = 0, stratify = y_data)

X_train_max, X_test_max, y_train, y_test = train_test_split(X_data_max, y_data, test_size = 0.2,
                                                              random_state = 0, stratify = y_data)

#### RandomForest 분류 모델을 이용한 텍스트 감성 분석

In [81]:
from sklearn.ensemble import RandomForestClassifier

# n_estimators = 500
rf_mean500 = RandomForestClassifier(n_estimators = 500, random_state = 0)
rf_median500 = RandomForestClassifier(n_estimators = 500, random_state = 0)
rf_max500 = RandomForestClassifier(n_estimators = 500, random_state = 0)

##### 모델 학습

In [82]:
rf_mean500.fit(X_train_mean, y_train)

RandomForestClassifier(n_estimators=500, random_state=0)

In [83]:
y_pred_mean = rf_mean500.predict(X_test_mean)
print(y_pred)

[1 0 1 ... 1 1 1]


In [84]:
from sklearn.metrics import classification_report

report_mean = classification_report(y_test, y_pred_mean)
print(report_mean)

              precision    recall  f1-score   support

           0       0.88      0.83      0.85      2500
           1       0.84      0.88      0.86      2500

    accuracy                           0.86      5000
   macro avg       0.86      0.86      0.86      5000
weighted avg       0.86      0.86      0.86      5000



In [56]:
rf_median500.fit(X_train_median, y_train)

RandomForestClassifier(n_estimators=500, random_state=0)

In [59]:
y_pred_median = rf_median500.predict(X_test_median)

In [61]:
from sklearn.metrics import classification_report

report_median = classification_report(y_test, y_pred_median)
print(report_median)

              precision    recall  f1-score   support

           0       0.86      0.81      0.83      2500
           1       0.82      0.86      0.84      2500

    accuracy                           0.84      5000
   macro avg       0.84      0.84      0.84      5000
weighted avg       0.84      0.84      0.84      5000



In [ ]:
rf_max500.fit(X_train_max, y_train)

In [ ]:
y_pred_max = rf_max500.predict(X_test_max)

In [ ]:
from sklearn.metrics import classification_report

report_max = classification_report(y_test, y_pred_max)
print(report_max)

#### LightGBM 모델을 이용한 텍스트 감성 분석

##### 모델 생성 및 학습

In [ ]:
# # 최적화 할 기본 모델 생성
# lgbm = LGBMClassifier(random_state=0,
#                       bagging_seed=0,
#                       feature_fraction_seed=0,
#                       data_random_seed=0,
#                       drop_seed=0,
#                       verbose=-1)
# # 최적화 할 매개 변수 설정
# params = {'learning_rate':[0.05, 0.1, 0.2, 0.3],
#           'n_estimators':[300, 400, 500]
#           }
# # GridSearcCV 함수 호출, 객체 생성
# grid_lgbm = GridSearchCV(
#     estimator=lgbm,
#     param_grid=params,
#     scoring='accuracy',
#     cv=10
# )

In [85]:
from lightgbm import LGBMClassifier

# 매개변수 --> learning_rate : [0.05, 0.1, 0.2, 0.3], n_estimators : [300, 400, 500]
# learning_rate = 0.1, n_estimators = 500 --> 모델 생성
lgbm = LGBMClassifier(learning_rate = 0.1, n_estimators = 500,
                      random_state = 0, bagging_seed=0,
                      feature_fraction_seed=0, data_random_seed=0,
                      drop_seed=0, verbose=-1)

In [86]:
lgbm.fit(X_train_mean, y_train)

LGBMClassifier(bagging_seed=0, data_random_seed=0, drop_seed=0,
               feature_fraction_seed=0, n_estimators=500, random_state=0,
               verbose=-1)

In [87]:
y_pred_lgbm_mean = lgbm.predict(X_test_mean)
print(y_pred_lgbm_mean)

[1 0 1 ... 1 1 1]


In [88]:
from sklearn.metrics import classification_report

report_lgbm_mean = classification_report(y_test, y_pred_lgbm_mean)
print(report_lgbm_mean)

              precision    recall  f1-score   support

           0       0.89      0.87      0.88      2500
           1       0.88      0.89      0.88      2500

    accuracy                           0.88      5000
   macro avg       0.88      0.88      0.88      5000
weighted avg       0.88      0.88      0.88      5000

